## Fit of Neutron Reflectivity data from a polymer Brush

Suppose that we want to fit NR data of a polymer brushe at the Quartz/d-toluene interface. Polymer brushes have parabolic volume fraction profiles of the form: $\phi(z) = \phi(0) \left[ 1 - (\frac{z}{L})^n\right]$ where $\phi$ is the polymer volume fraction, $L$ is the brush extension, $n$ the exponent and $z$ the distance from the substrate.

First let's import _anaklasis.ref_

In [ ]:
from anaklasis import ref

We name the project as 'Brush_fit'. All results will be exported in a folder with this name. Then we import the experimental curve and we set input units to Angstrom.

In [ ]:
project='Brush_fit'
in_file=['DA34424a.dat']
units=['A'] # Angstrom units

then we set the details of the fit. Note that we will perform MCMC sampling.

In [ ]:
fit_mode=0 # linear
fit_weight=[1] 
method = 'mcmc' # Markov Chain Monte Carlo

We define a single model where we add the Quartz semi-infinite fronting and a thin water layer that is always present on the substrate in these measurements. A global parameter (that will be defined later) is associated with the thickness of this thin water layer. All other layer parameters are fixed.

In [ ]:
model=[
	# Re_sld Im_sld thk rough solv description
	[  4.32e-6, 0.0, 0, 5, 0.0, 'Quartz'],
	[ -0.56e-6, 0.0, 'p3', 3, 0.0, 'water'],
	]

We will now append 20 slices representing the polymer brush. These layers will all have the same polymer sld and a varying solvent volume fraction ($1-\phi(z)$). For this reason we first define a symbolic expression for the solvent volume fraction.

In [ ]:
expr='1-p0+(p0/(p1**p2))*((p1/20)*(n-1.5))**p2'

where p0, p1, p2 are global parameters (we will define them later) representing $\phi(0)$, $L$ and the exponent. $n$ is the layer number that for our brush layer runs from 2 to 21. So the expression $(p1/20)*(n-1.5)$ gives the $z$ distance of each slice from the substrate. So now we append the slices of the brush in the model _list_ using a _for loop_.

In [ ]:
for i in range(20): 
	model.append([ 1.41e-6, 0.0,'p1/20', 'p1/40', expr, 'layer'])

All slices have the same sld, same thickness and we also apply a roughness (half slice thickness) to smooth the profile. The volume fraction profile is set according to the above defined symbolic expression.

We also add at the end of the model the semi-infinite backing (d-toluene)

In [ ]:
model.append([  5.86e-6, 0.0,  0, 0, 1.0, 'd-toluene'])

Now that the model _list_ is complete and since we have a single model (patch) of 100% coverage in our system, we define the _patches_ and _system_ _lists_. 

In [ ]:
patches=[1.0]
system=[model]

Now we may define the global parameters. Note that the global parameters can be of type uniform or normal. For a uniform parameter we assume that the parameter is uniformly distributed in the given min/max bounds. For a normal parameter we assume that the parameter is normally distributed with the defined mean and standard deviation.

In [ ]:
global_param = [
    # param  min  max  description  type='uniform'
    # param  mean  sd  description  type='normal'
	['p0', 0.02, 0.20, 'phi0','uniform'],
	['p1', 100, 700, 'brush_length','uniform'],
	['p2', 1, 4, 'exponent','uniform'],
	['p3', 5, 2, 'water thickness','normal'],
	]

We have no multi-parameters or constraints to declare

In [ ]:
multi_param = [] # no multi-parameters
constraints = [] # no constraints

we also define the instrumental parameters. Background and scale are set as fixed.

In [ ]:
resolution=[0.06] # dQ/Q=6%
background = [[0.0,0.0,'uniform']] #background corrected data
scale = [[1.0,1.0,'uniform']] #data correctly scaled

We finally call the _ref.fit_ function and store the results in the _dictionary_ _res_. Note that we set _experror=True_ since our input data contain the $dR(Q)$ errors. 

In [ ]:
res = ref.fit(project, in_file, units, fit_mode,fit_weight,method,resolution,patches, system,
global_param,multi_param, constraints,background,scale,experror=True, plot=True,fast=True)

_anaklasis_ automatically exports a number of plots for the reflectivity and profiles. A corner plot from the MCMC sampling is exported. If we want to plot manually the volume fraction profile and "zoom" close to the region of the variation of the brush profile we may use the results stored in the res _dictionary_. 

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.ylim([0.0,0.102])
plt.plot(res[("solvent")][:,0],1-res[("solvent")][:,1],color='black')
plt.xlabel(r'$z(\AA)$')
plt.ylabel(r'$brush volume fraction$')
plt.draw()